In [32]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *

In [33]:
### get file names ###
env = 'local'
files = os.listdir('./database/')

for i in range(len(files)):
    print(i, files[i])

0 Ag_100_u2_RPBE.traj
1 Ag_100_u2_RPBE_all.traj
2 Ag_100_u2_RPBE_no001_CO_n1_d9.traj
3 Ag_100_u2_RPBE_no001_CO_n1_d9__.traj
4 Ag_100_u2_RPBE_no001_CO_n1_d9___all.traj
5 Ag_100_u2_RPBE_no001_CO_n1_d9_all.traj
6 Ag_100_u2_RPBE_no002_CO_n1_d9.traj
7 Ag_100_u2_RPBE_no002_CO_n1_d9__.traj
8 Ag_100_u2_RPBE_no002_CO_n1_d9___all.traj
9 Ag_100_u2_RPBE_no002_CO_n1_d9_all.traj
10 Ag_100_u2_RPBE_no002_CO_n2_d3.traj
11 Ag_100_u2_RPBE_no002_CO_n2_d3__.traj
12 Ag_100_u2_RPBE_no002_CO_n2_d3___all.traj
13 Ag_100_u2_RPBE_no002_CO_n2_d3_all.traj
14 Ag_100_u2_RPBE_no003_CO_n1_d9.traj
15 Ag_100_u2_RPBE_no003_CO_n1_d9__.traj
16 Ag_100_u2_RPBE_no003_CO_n1_d9___all.traj
17 Ag_100_u2_RPBE_no003_CO_n1_d9_all.traj
18 Ag_100_u2_RPBE_no003_CO_n3_d3.traj
19 Ag_100_u2_RPBE_no003_CO_n3_d3__.traj
20 Ag_100_u2_RPBE_no003_CO_n3_d3___all.traj
21 Ag_100_u2_RPBE_no003_CO_n3_d3_all.traj
22 Ag_100_u2_RPBE_no004_CO_n4_d3.traj
23 Ag_100_u2_RPBE_no004_CO_n4_d3__.traj
24 Ag_100_u2_RPBE_no004_CO_n4_d3___all.traj
25 Ag_100_u2_RPBE_

1389 CoPt3_100_u2_RPBE_no219_CO_n5_d3__.traj
1390 CoPt3_100_u2_RPBE_no219_CO_n5_d3___all.traj
1391 CoPt3_100_u2_RPBE_no219_CO_n5_d3_all.traj
1392 CoPt3_100_u2_RPBE_no224_CO_n4_d3.traj
1393 CoPt3_100_u2_RPBE_no224_CO_n4_d3__.traj
1394 CoPt3_100_u2_RPBE_no224_CO_n4_d3___all.traj
1395 CoPt3_100_u2_RPBE_no224_CO_n4_d3_all.traj
1396 CoPt3_100_u2_RPBE_no225_CO_n4_d3.traj
1397 CoPt3_100_u2_RPBE_no225_CO_n4_d3__.traj
1398 CoPt3_100_u2_RPBE_no225_CO_n4_d3___all.traj
1399 CoPt3_100_u2_RPBE_no225_CO_n4_d3_all.traj
1400 CoPt3_100_u2_RPBE_no231_CO_n3_d3.traj
1401 CoPt3_100_u2_RPBE_no231_CO_n3_d3__.traj
1402 CoPt3_100_u2_RPBE_no231_CO_n3_d3___all.traj
1403 CoPt3_100_u2_RPBE_no231_CO_n3_d3_all.traj
1404 CoPt3_100_u2_RPBE_no232_CO_n3_d3.traj
1405 CoPt3_100_u2_RPBE_no232_CO_n3_d3__.traj
1406 CoPt3_100_u2_RPBE_no232_CO_n3_d3___all.traj
1407 CoPt3_100_u2_RPBE_no232_CO_n3_d3_all.traj
1408 CoPt3_100_u2_RPBE_no233_CO_n3_d3.traj
1409 CoPt3_100_u2_RPBE_no233_CO_n3_d3__.traj
1410 CoPt3_100_u2_RPBE_no233_CO_n3_

2820 Pd_110_u2_RPBE.traj
2821 Pd_110_u2_RPBE_all.traj
2822 Pd_110_u2_RPBE_no001_CO_n1_d9.traj
2823 Pd_110_u2_RPBE_no001_CO_n1_d9__.traj
2824 Pd_110_u2_RPBE_no001_CO_n1_d9___all.traj
2825 Pd_110_u2_RPBE_no001_CO_n1_d9_all.traj
2826 Pd_110_u2_RPBE_no002_CO_n1_d9.traj
2827 Pd_110_u2_RPBE_no002_CO_n1_d9__.traj
2828 Pd_110_u2_RPBE_no002_CO_n1_d9___all.traj
2829 Pd_110_u2_RPBE_no002_CO_n1_d9_all.traj
2830 Pd_110_u2_RPBE_no002_CO_n2_d5.traj
2831 Pd_110_u2_RPBE_no002_CO_n2_d5__.traj
2832 Pd_110_u2_RPBE_no002_CO_n2_d5___all.traj
2833 Pd_110_u2_RPBE_no002_CO_n2_d5_all.traj
2834 Pd_110_u2_RPBE_no003_CO_n1_d9.traj
2835 Pd_110_u2_RPBE_no003_CO_n1_d9__.traj
2836 Pd_110_u2_RPBE_no003_CO_n1_d9___all.traj
2837 Pd_110_u2_RPBE_no003_CO_n1_d9_all.traj
2838 Pd_110_u2_RPBE_no003_CO_n3_d3.traj
2839 Pd_110_u2_RPBE_no003_CO_n3_d3__.traj
2840 Pd_110_u2_RPBE_no003_CO_n3_d3___all.traj
2841 Pd_110_u2_RPBE_no003_CO_n3_d3_all.traj
2842 Pd_110_u2_RPBE_no004_CO_n1_d9.traj
2843 Pd_110_u2_RPBE_no004_CO_n1_d9__.traj
2844

In [34]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

for i in range(len(files)):
    if 'E_no' in files[i]:
        if '__' not in files[i]:
            if 'all' not in files[i]:
                filenames.append(files[i])
                print(i, files[i])

2 Ag_100_u2_RPBE_no001_CO_n1_d9.traj
6 Ag_100_u2_RPBE_no002_CO_n1_d9.traj
10 Ag_100_u2_RPBE_no002_CO_n2_d3.traj
14 Ag_100_u2_RPBE_no003_CO_n1_d9.traj
18 Ag_100_u2_RPBE_no003_CO_n3_d3.traj
22 Ag_100_u2_RPBE_no004_CO_n4_d3.traj
26 Ag_100_u2_RPBE_no005_CO_n3_d3.traj
30 Ag_100_u2_RPBE_no006_CO_n4_d3.traj
34 Ag_100_u2_RPBE_no007_CO_n2_d3.traj
38 Ag_100_u2_RPBE_no008_CO_n2_d3.traj
42 Ag_100_u2_RPBE_no009_CO_n3_d3.traj
46 Ag_100_u2_RPBE_no011_CO_n2_d3.traj
50 Ag_100_u2_RPBE_no012_CO_n3_d3.traj
54 Ag_100_u2_RPBE_no013_CO_n4_d3.traj
58 Ag_100_u2_RPBE_no014_CO_n2_d3.traj
62 Ag_100_u2_RPBE_no015_CO_n2_d3.traj
68 Ag_110_u2_RPBE_no001_CO_n1_d9.traj
72 Ag_110_u2_RPBE_no002_CO_n1_d9.traj
76 Ag_110_u2_RPBE_no002_CO_n2_d6.traj
80 Ag_110_u2_RPBE_no003_CO_n1_d9.traj
84 Ag_110_u2_RPBE_no003_CO_n3_d3.traj
88 Ag_110_u2_RPBE_no004_CO_n1_d9.traj
92 Ag_110_u2_RPBE_no005_CO_n3_d3.traj
96 Ag_110_u2_RPBE_no006_CO_n4_d3.traj
100 Ag_110_u2_RPBE_no007_CO_n2_d6.traj
104 Ag_110_u2_RPBE_no008_CO_n3_d3.traj
108 Ag_110_u

3138 Pd_211_u2_RPBE_no010_CO_n1_d9.traj
3142 Pd_211_u2_RPBE_no010_CO_n2_d3.traj
3146 Pd_211_u2_RPBE_no012_CO_n2_d3.traj
3150 Pd_211_u2_RPBE_no013_CO_n3_d3.traj
3154 Pd_211_u2_RPBE_no014_CO_n4_d3.traj
3158 Pd_211_u2_RPBE_no015_CO_n2_d3.traj
3162 Pd_211_u2_RPBE_no016_CO_n3_d3.traj
3166 Pd_211_u2_RPBE_no018_CO_n2_d3.traj
3172 Pt_100_u2_RPBE_no001_CO_n1_d9.traj
3176 Pt_100_u2_RPBE_no002_CO_n1_d9.traj
3180 Pt_100_u2_RPBE_no002_CO_n2_d3.traj
3184 Pt_100_u2_RPBE_no003_CO_n1_d9.traj
3188 Pt_100_u2_RPBE_no003_CO_n3_d3.traj
3192 Pt_100_u2_RPBE_no004_CO_n4_d3.traj
3196 Pt_110_u2_RPBE_no001_CO_n1_d9.traj
3200 Pt_110_u2_RPBE_no002_CO_n1_d9.traj
3204 Pt_110_u2_RPBE_no003_CO_n1_d9.traj
3208 Pt_110_u2_RPBE_no004_CO_n1_d9.traj
3214 Pt_111_u2_RPBE_no001_CO_n1_d9.traj
3218 Pt_111_u2_RPBE_no002_CO_n1_d9.traj
3222 Pt_111_u2_RPBE_no003_CO_n1_d9.traj
3226 Pt_111_u2_RPBE_no004_CO_n1_d9.traj
3232 Pt_211_u2_RPBE_no001_CO_n1_d9.traj
3236 Pt_211_u2_RPBE_no002_CO_n1_d9.traj
3240 Pt_211_u2_RPBE_no003_CO_n1_d9.traj


In [35]:
# ### DELETE ###
# for file in filenames:
#     try:
#         A = make_database(file)
#         A.delete_from_database()
#     except:
#         print(file, 'Some error occured.')

In [36]:
### ADD and UPDATE ENERGY ###
for file in filenames:
    try:
        A = make_database(file)
        A.add_to_database()
        A.update_Energy()
        A.update_adsorbates_correlation()
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n2_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_100_u2_RPB

E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ag_110_u2_RPBE_no024_CO_n4_d5.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ag_110_u2_RPBE_no025_CO_n2_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ag_110_u2_RPBE_no026_CO_n3_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ag_110_u2_RPBE_no027_CO_n4_d5.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ag_110_u2_RPBE_no028_CO_n2_d3.traj
Already in data

E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ag_111_u2_RPBE_no003_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_111_u2_RPBE_no003_CO_n3_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ag_111_u2_RPBE_no004_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_111_u2_RPBE_no004_CO_n4_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
----------------------------

E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Al_100_u2_RPBE_no003_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Al_100_u2_RPBE_no003_CO_n3_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Al_100_u2_RPBE_no004_CO_n4_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Al_100_u2_RPBE_no005_CO_n3_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Al_100_u2

E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Al_110_u2_RPBE_no018_CO_n4_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Al_110_u2_RPBE_no019_CO_n2_d6.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Al_110_u2_RPBE_no020_CO_n2_d6.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Al_110_u2_RPBE_no022_CO_n2_d6.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Al_110_u2_RPBE_no023_CO_n3_d3.traj
Already in data

...

E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ni_111_u2_RPBE_no009_CO_n2_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ni_111_u2_RPBE_no010_CO_n3_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ni_111_u2_RPBE_no011_CO_n4_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ni_111_u2_RPBE_no012_CO_n2_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Ni_111_u2_RPBE_no013_CO_n2_d3.traj
Already in data

E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_100_u2_RPBE_no008_CO_n2_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_100_u2_RPBE_no009_CO_n3_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_100_u2_RPBE_no011_CO_n2_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_100_u2_RPBE_no012_CO_n3_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_100_u2_RPBE_no013_CO_n4_d3.traj
Already in data

E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_110_u2_RPBE_no024_CO_n2_d5.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_110_u2_RPBE_no026_CO_n2_d5.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_110_u2_RPBE_no027_CO_n3_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_110_u2_RPBE_no028_CO_n4_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_110_u2_RPBE_no029_CO_n3_d3.traj
Already in data

E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_211_u2_RPBE_no003_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Pd_211_u2_RPBE_no003_CO_n3_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
-----------------------------------------------------------
Pd_211_u2_RPBE_no004_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Pd_211_u2_RPBE_no004_CO_n4_d3.traj
Already in database
E_each_ads and E_residue/suratom updated.
adsorbates correlation already in database.
----------------------------

-----------------------------------------------------------
Rh_111_u2_RPBE_no001_CO_n1_d8.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Rh_111_u2_RPBE_no002_CO_n1_d8.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Rh_111_u2_RPBE_no003_CO_n1_d8.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Rh_111_u2_RPBE_no004_CO_n1_d8.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
--------------------------------

In [40]:
for file in filenames:
    try:
        A = make_database(file)
        data = A.check_database()
        print(data['name'], data['minimum_distance'], data['ads_dist2'], data['ads_dist3'])
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Ag_100_u2_RPBE_no001_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Ag_100_u2_RPBE_no002_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
Ag_100_u2_RPBE_no003_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no001_CO_n1_d9.traj
Ag_110_u2_RPBE_no001_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no002_CO_n1_d9.traj
Ag_110_u2_RPBE_no002_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no003_CO_n1_d9.traj
Ag_110_u2_RPBE_no003_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no004_CO_n1_d9.traj
Ag_110_u2_RPBE_no004_CO_n1_d9.traj 

-----------------------------------------------------------
Au_111_u2_RPBE_no004_CO_n1_d9.traj
Au_111_u2_RPBE_no004_CO_n1_d9.traj None None None
-----------------------------------------------------------
Au_111_u2_RPBE_no004_CO_n4_d3.traj
Au_111_u2_RPBE_no004_CO_n4_d3.traj 3 0.0 2.6666666666666665
-----------------------------------------------------------
Au_111_u2_RPBE_no005_CO_n3_d3.traj
Au_111_u2_RPBE_no005_CO_n3_d3.traj 3 0.0 1.3888888888888888
-----------------------------------------------------------
Au_111_u2_RPBE_no006_CO_n4_d3.traj
Au_111_u2_RPBE_no006_CO_n4_d3.traj 2 1.0 0.1111111111111111
-----------------------------------------------------------
Au_111_u2_RPBE_no007_CO_n2_d3.traj
Au_111_u2_RPBE_no007_CO_n2_d3.traj 3 0.0 0.5961199294532626
-----------------------------------------------------------
Au_111_u2_RPBE_no008_CO_n3_d3.traj
Au_111_u2_RPBE_no008_CO_n3_d3.traj 2 0.6666666666666666 1.037037037037037
-----------------------------------------------------------
Au_111

-----------------------------------------------------------
Cu_211_u2_RPBE_no002_CO_n1_d8.traj
Cu_211_u2_RPBE_no002_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no003_CO_n1_d8.traj
Cu_211_u2_RPBE_no003_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no004_CO_n1_d8.traj
Cu_211_u2_RPBE_no004_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no005_CO_n1_d8.traj
Cu_211_u2_RPBE_no005_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no006_CO_n1_d8.traj
Cu_211_u2_RPBE_no006_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no007_CO_n1_d8.traj
Cu_211_u2_RPBE_no007_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no008_CO_n1_d8.traj
Cu_211_u2_RPBE_no008_CO_n1_d8.traj 

Rh_100_u2_RPBE_no003_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_110_u2_RPBE_no001_CO_n1_d8.traj
Rh_110_u2_RPBE_no001_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_110_u2_RPBE_no002_CO_n1_d8.traj
Rh_110_u2_RPBE_no002_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_110_u2_RPBE_no003_CO_n1_d8.traj
Rh_110_u2_RPBE_no003_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_110_u2_RPBE_no004_CO_n1_d8.traj
Rh_110_u2_RPBE_no004_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_111_u2_RPBE_no001_CO_n1_d8.traj
Rh_111_u2_RPBE_no001_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_111_u2_RPBE_no002_CO_n1_d8.traj
Rh_111_u2_RPBE_no002_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_111_u2_RPBE_no003

In [19]:
# for file in filenames:
#     try:
#         A = make_database(file)
#         print(A.check_database()['minimum_distance'])
#     except:
#         print(file, 'Some error occured.')

In [20]:
file = 'Cu_100_u2_RPBE_no006_CO_n4_d3.traj'
A = make_database(file)
A.check_database()

No file named Cu_100_u2_RPBE_no006_CO_n4_d3.traj in database
-----------------------------------------------------------
Cu_100_u2_RPBE_no006_CO_n4_d3.traj
Not in database.


In [21]:
query('Cu_110_u2_RPBE_no003_CO_n3_d3.traj','local')

No file named Cu_110_u2_RPBE_no003_CO_n3_d3.traj in database
